In [1]:
import requests
from bs4 import BeautifulSoup as bs
from pprint import pprint
import pandas as pd
import datetime

mainRef = "http://vsesoki.ru"




In [2]:
def getPrices(ref):
    responce = requests.get(ref).text
    html = bs(responce, "lxml")

    tags = html.find_all("div", class_="prod-block")
    prices = {}
    for tag in tags:
        name = tag.find("p", class_="title").getText().strip()
        price = tag.find("p", class_="price").getText().strip(" руб.")
        if (int(price) != 0):
            prices[name] = price
    return(prices)

In [3]:
def getPageRefs(ref):
    responce = requests.get(ref).text
    html = bs(responce, "lxml")
    
    pageNumTags = html.find("div", class_="pagination fr")
    if pageNumTags == None:
        return([ref])
    
    pageNums = pageNumTags.getText().split()
    lastPageNum = int(pageNums[-1])

    refs = [ref + "?page=" + str(x) for x in range(2,lastPageNum + 1)]
    refs = [ref] + refs
    return(refs)

In [4]:
def getAllPrices(refs):
    allPrices = {}

    for pageRef in refs:
        prices = getPrices(pageRef)
        allPrices.update(prices)
    return(allPrices)

In [5]:
def printToFile(allPrices, fname):

    data = pd.DataFrame(list(allPrices.items()))
    data.columns = ["name","price"]
    
    data.sort_values("name").to_csv(fname, index = False, sep = ";", encoding='cp1251')


In [6]:
def getAll(ref):
        
    refs = getPageRefs(ref)
    
    allPrices = getAllPrices(refs)
    
    return(allPrices)

In [7]:
def getCtalogRefs(ref):
    refs = []
    
    responce = requests.get(ref).text
    html = bs(responce, "lxml")
    
    colBlockTags = html.findAll("div", class_="col-block")
    for tag in colBlockTags:
        ptag = tag.find("p")
        atag = ptag.find("a")
        refs.append(ref + atag['href'])
    
    return(refs)

In [8]:
if __name__ == '__main__':    
    now = datetime.datetime.now()
    fname = "VseSoki_" + now.strftime("%d-%m-%Y") + ".csv"
    
    catRefs = getCtalogRefs(mainRef)
    
    allPrices = {}
    
    for ref in catRefs:
        print(ref)
        allPrices.update(getAll(ref))
    
    printToFile(allPrices, fname)
    
    

http://vsesoki.ru/collection/shnekovye-sokovyzhimalki
http://vsesoki.ru/collection/blendery
http://vsesoki.ru/collection/degidratory
http://vsesoki.ru/collection/melnitsy
http://vsesoki.ru/collection/maslopressy
http://vsesoki.ru/collection/proraschivateli
http://vsesoki.ru/collection/Zdorovoe-pitanie
http://vsesoki.ru/collection/chistaya-voda
http://vsesoki.ru/collection/vakuumnye-upakovschiki
http://vsesoki.ru/collection/hlebopechki
http://vsesoki.ru/collection/komplekty
http://vsesoki.ru/collection/podarki-za-otzyv
http://vsesoki.ru/collection/dlya-kuhni-i-zdorovya
http://vsesoki.ru/collection/kommercheskoe-oborudovanie
http://vsesoki.ru/collection/massazhery
http://vsesoki.ru/collection/brendy
http://vsesoki.ru/collection/zapchasti
http://vsesoki.ru/collection/books
